# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
Big4='NHL'
# nhl_df.head()
print(cities.shape)
cities.head()


In [ ]:
cities['NHL']

In [ ]:
team = cities['NHL'].str.extract('([A-Z]{0,2}[a-z0-9]*\\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')

team['Metropolitan area']=cities['Metropolitan area']
team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
team= pd.merge(team, cities, how='left', on ='Metropolitan area').iloc[:, 1:4]
team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
team
# team[team['Metropolitan area']=='Columbus']

In [ ]:
nhl=pd.read_csv("assets/nhl.csv")
nhl = nhl[nhl['year'] == 2018] # Keep only 2018 season
nhl['team'] = nhl['team'].str.replace(r'\*',"", regex=True) # Clean NHL Team Names (Remove "*")
nhl = nhl[['team','W','L']]

nhl
# nhl[nhl['team']=='Blue Jackets']

In [ ]:
dropList=[]
for i in range(nhl.shape[0]):
    row=nhl.iloc[i]
    if row['team']==row['W'] and row['L']==row['W']:
        print(row['team'],'will be dropped!')
        dropList.append(i)
nhl=nhl.drop(dropList)
print(nhl.shape)
nhl

In [ ]:
nhl['team'] = nhl['team'].str.replace('[\\w.]* ','', regex=True) #remove until the last word
nhl = nhl.astype({'team': str,'W': int, 'L': int})
nhl['WLRatio'] = nhl['W']/(nhl['W']+nhl['L'])

print(nhl.shape)
nhl
# team[team['Metropolitan area']=='Columbus']
# nhl[nhl['team']=='Blue Jackets']

In [ ]:
team['team']=team['team'].str.replace('[\\w]*\\ ','')
team['team'].shape

In [ ]:
merge=pd.merge(team,nhl,'outer', on = 'team')
# merge=merge.groupby('Metropolitan area').agg({'WLRatio': 'mean', 'Population': 'mean'})
print(merge.shape)
merge
# merge.loc[merge['Metropolitan area']==' Devils']

In [ ]:
merge=merge.groupby('Metropolitan area').agg({'WLRatio': 'mean', 'Population': 'mean'})
print(merge.shape)
merge

In [ ]:
population_by_region = merge['Population']
win_loss_by_region = merge['WLRatio']

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

stats.pearsonr(population_by_region, win_loss_by_region)

In [36]:
def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))

nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,area
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Tampa Bay Area
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Toronto
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Miami–Fort Lauderdale
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Detroit
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL,Montreal
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL,Ottawa
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Buffalo
10,Washington Capitals,82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL,"Washington, D.C."
11,Pittsburgh Penguins,82,47,29,6,100,.610,272,250,0.23,-0.04,.573,45,2018,NHL,Pittsburgh


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

def nhl_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


def get_nhl_data():
    return out_df

In [55]:
def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def clear_nba_data(string1):
    if re.search(r'\* \([0-9]*\)| \([0-9]*\)', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\* \([0-9]*\)| \([0-9]*\)', string1).group(), '')


def get_area(team):
    for each in list(nba_cities.index.values):
        if team in each:
            return nba_cities.at[each, 'Metropolitan area']


nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA'] = cities['NBA'].apply(lambda x: clear_data(x))
nba_cities = cities[['Metropolitan area', 'NBA']].set_index('NBA')
nba_cities = nba_cities.drop(['—', ''], axis=0)
nba_df = nba_df[nba_df['year'] == 2018] # get only 2018 stats
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
nba_df['team'] = nba_df['team'].str.replace(r'\*?\s*\(\d+\)$',"", regex=True)
nba_df['area'] = nba_df['team'].apply(lambda x: x.split(" ")[-1])
nba_df['area'] = nba_df['area'].apply(lambda x: get_area(x))
nba_df
# nba_cities

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,area
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Toronto
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Boston
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,Philadelphia
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cleveland
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Indianapolis
5,Miami Heat,44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA,Miami–Fort Lauderdale
6,Milwaukee Bucks,44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA,Milwaukee
7,Washington Wizards,43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA,"Washington, D.C."
8,Detroit Pistons,39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA,Detroit
9,Charlotte Hornets,36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA,Charlotte


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
def clear_nba_data(string1):
    if re.search(r'\* \([0-9]*\)| \([0-9]*\)', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\* \([0-9]*\)| \([0-9]*\)', string1).group(), '')

def get_area(team):
    for each in list(nba_cities.index.values):
        if team in each:
            return nba_cities.at[each, 'Metropolitan area']
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values